<a class = "anchor" id = "top"></a>

# NLP- English
## Source: Twitter
---
### Authors: Gordon Amoako, Zan Sadiq
---

Table of Contents:
* [Data](#data)
* [Pre-Processing](#eda)
* [Networking](#network)
* [NLP](#nlp)
* [Conclusion](#end)
---

## Data <a class = "anchor" id = "data"></a>

In [152]:
# Import libraries
import pandas as pd
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType, ArrayType, FloatType
import re
import string
from pyspark.sql.functions import udf, col, size, lit, explode, isnan, when, count, min, max, struct
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, IndexToString, StringIndexer, VectorIndexer, CountVectorizer
from collections import Counter
import networkx as nx

In [43]:
# Function to get hashtags
def extract_hashtags(x):
    
    hashtag_list = []
      
    # splitting the text into words
    for word in x.split():
          
        # checking the first charcter of every word
        if word[0] == '#':
              
            # adding the word to the hashtag_list
            hashtag_list.append(word[1:])
      
    return hashtag_list

# Function to process text
def clean_tweet(tweet):

    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet.lower()).split())

# Function to further process text
def more_cleaning(tweet):
    
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet)
    tweet = ''.join([i for i in tweet if not i.isdigit()])
    tweet = " ".join(re.split("\s+", tweet, flags = re.UNICODE))
    
    return tweet

# Function to filter pos
def filter_pos(x):

    x = nltk.pos_tag(x)
    x = [i[0] for i in x if i[1].startswith(('N', 'A', 'J'))]

    return x

In [3]:
# Get wd
os.getcwd()

'/home/dataguy/Documents'

In [4]:
# Initialize spark
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.getOrCreate()

22/08/15 08:10:11 WARN Utils: Your hostname, computer resolves to a loopback address: 127.0.1.1; using 192.168.1.159 instead (on interface wlp0s20f3)
22/08/15 08:10:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/15 08:10:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/15 08:10:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/15 08:10:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/15 08:10:11 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/08/15 08:10:11 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/08/15 08:10:11 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/08/15 08:10:11 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/08/15 08:10:11 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
22/08/15 08:10:11 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.


In [5]:
df = spark.read.json('/home/dataguy/news.json')

22/08/15 08:10:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
df.printSchema()

root
 |-- _type: string (nullable = true)
 |-- cashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- content: string (nullable = true)
 |-- conversationId: long (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- _type: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- date: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: long (nullable = true)
 |-- inReplyToTweetId: long (nullable = true)
 |-- inReplyToUser: struct (nullable = true)
 |    |-- _type: string (nullable = true)
 |    |-- created: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- descriptionUrls: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |  

In [7]:
df.count()

2828592

In [8]:
# Show nulls
#cols = df.columns
#df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in cols]).show()

In [12]:
# Inspect
min_date, max_date = df.select(min("date"), max("date")).first()
print(f"Min Date- {min_date}")
print(f"Max Date- {max_date}")

Min Date- 2022-08-04T11:38:14+00:00
Max Date- 2022-08-14T15:51:17+00:00


[Back to top...](#top)

## Pre-Processing <a class = "anchor" id = "eda"></a>

In [16]:
df1 = df.withColumn('username', col('user.username')).withColumn('country', col('place.country')).withColumn('country_cd', col('place.countryCode')).drop('user', 'coordinates', 'place')

In [18]:
df1.groupBy('country').count().show()

+--------------------+-------+
|             country|  count|
+--------------------+-------+
|              Russia|     99|
|              Sweden|     93|
|     The Netherlands|    174|
|              Guyana|      4|
|            Malaysia|    150|
|           Singapore|     73|
|              Turkey|    173|
|                Iraq|      9|
|             Germany|    396|
|              France|    400|
|              Greece|    123|
|           Sri Lanka|     96|
|Republic of the P...|    312|
|              Taiwan|     49|
|                null|2781648|
|           Argentina|    122|
|             Belgium|     56|
|               Qatar|     25|
|               Ghana|    262|
|       United States|  17953|
+--------------------+-------+
only showing top 20 rows



In [41]:
df1.select('inReplyToUser').take(1)

[Row(inReplyToUser=Row(_type='snscrape.modules.twitter.User', created=None, description=None, descriptionUrls=None, displayname='NYT Politics', favouritesCount=None, followersCount=None, friendsCount=None, id=14434063, label=None, linkTcourl=None, linkUrl=None, listedCount=None, location=None, mediaCount=None, profileBannerUrl=None, profileImageUrl=None, protected=None, rawDescription=None, statusesCount=None, url='https://twitter.com/nytpolitics', username='nytpolitics', verified=None))]

In [42]:
df1.select('mentionedUsers').take(1)

[Row(mentionedUsers=[Row(_type='snscrape.modules.twitter.User', created=None, description=None, descriptionUrls=None, displayname='NYT Politics', favouritesCount=None, followersCount=None, friendsCount=None, id=14434063, label=None, linkTcourl=None, linkUrl=None, listedCount=None, location=None, mediaCount=None, profileBannerUrl=None, profileImageUrl=None, protected=None, rawDescription=None, statusesCount=None, url='https://twitter.com/nytpolitics', username='nytpolitics', verified=None)])]

In [70]:
df1.filter(~col('quotedTweet').isNull()).select('quotedTweet.user.username').take(1)

[Row(username='AmarUjalaNews')]

In [78]:
df1.filter(~col('retweetedTweet').isNull()).take(1) #.select('retweetedTweet.user.username').take(1)

[]

In [81]:
df2 = df1.withColumn('quoted', col('quotedTweet.user.username')).drop('quotedTweet').withColumn('mentions', col('mentionedUsers.username')).withColumn('reply_to', col('inReplyToUser.username')).drop('inReplyToUser', 'mentionedUsers').filter("country == 'United States'").toPandas().drop(['renderedContent', 'id', 'media', 'outlinks', '_type', 'cashtags', 'conversationId', 'inReplyToTweetId', 'source', 'sourceUrl', 'sourceLabel', 'tcooutlinks', 'url', 'country', 'lang', 'retweetedTweet'], axis = 1).set_index(pd.DatetimeIndex(df2['date'])).drop('date', axis = 1)

In [82]:
df2.head()

,content,hashtags,likeCount,quoteCount,replyCount,retweetCount,username,country_cd,quoted,mentions,reply_to
date,,,,,,,,,,,
2022-08-14 15:48:01+00:00,@VaticanNews @Synod_va a day ago Vatican news ...,None,0,0,0,0,JmDV808,US,None,"[VaticanNews, Synod_va]",JmDV808
2022-08-14 15:47:15+00:00,"This is great news, been looking for an excuse...",None,0,0,0,0,BigCountryPhil,US,SarahTheHaider,None,None
2022-08-14 15:46:08+00:00,One thing about Ghanaian men they will marry. ...,None,1,0,0,1,VickieRemoe,US,None,None,None
2022-08-14 15:42:50+00:00,News Release from #UCLA on ArcStorm 2.0 study ...,[UCLA],0,0,0,0,EpipilotIDWx,US,None,None,None
2022-08-14 15:42:41+00:00,WGN is just a trash ass news station. But she ...,None,1,0,0,0,joshuacharles__,US,A_Daneshzadeh,None,None


In [67]:
df2.shape[0]

17953

In [83]:
df2.to_csv('us_processed_tweets.csv')

[Back to top...](#top)

## Networking <a class = "anchor" id = "network"></a>

In [110]:
# Separate users
nodes = [i for i in df2['username'].unique()]

In [111]:
len(nodes)

11965

In [112]:
# Add user activites
for i in df2['quoted']:
    
    if i is not None:
        
        nodes.append(i)
        
for x in df2['mentions']:
    
    if x is not None:
        
        for y in x:
            
            nodes.append(y)
            
for z in df2['reply_to']:
    
    if z is not None:
        
        nodes.append(z)

In [116]:
nodes = [i for i in set(nodes)]
edges = []

In [117]:
len(nodes)

26545

In [118]:
# Select data
replies = df2.dropna(subset = ['reply_to'])
mentions = df2.dropna(subset = ['mentions'])
quotes = df2.dropna(subset = ['quoted']) 

In [119]:
# Iterate the data to add replies
for idx, row in replies.iterrows():
    
    edges.append((row['username'], row['reply_to']))

In [120]:
# Iterate the data to add mentions
for idx, row in mentions.iterrows():
    
    for entity in row['mentions']:
        
        edges.append((row['username'], entity))

In [121]:
# Iterate the data to add quotes
for idx, row in quotes.iterrows():
    
    edges.append((row['username'], row['quoted']))

In [122]:
nodes[0:5]

['BlairAcademy_NJ', 'Blackmagic_News', 'blutmer', 'cpmorgan03', 'edinaschools']

In [133]:
edges[0:5]

[('JmDV808', 'JmDV808'),
 ('howserob', 'narosenblum'),
 ('MetroGram', 'tomiahonen'),
 ('SicardNathaniel', 'conquest314'),
 ('122644', 'Brink_Thinker')]

In [138]:
edges = pd.DataFrame(edges, columns = ['source', 'target'])

In [145]:
# Calculate edge weights
weights = edges.groupby(['source', 'target']).size().reset_index().rename({0: 'weight'}, axis = 1)

In [146]:
weights.head(n = 20)

,source,target,weight
0,00000000,retrofightgamer,2
1,00_Barbara_00,00_Barbara_00,1
2,00_Barbara_00,sampson_dog,1
3,012488BWF884210,ChamberFayette,1
4,012488BWF884210,FCR_News,1
5,012488BWF884210,Fayettecountydd,1
6,012488BWF884210,fayettesheriff1,1
7,012488BWF884210,goblinshoes,1
8,06ers,11Yanks,2
9,07blacksummer,RonFilipkowski,2


In [149]:
weights.to_csv('weighted_twitter_news_networks.csv', index = False)

In [153]:
g = nx.from_pandas_edgelist(weights)

[Back to top...](#top)

## NLP: <a class = "anchor" id = "nlp"></a>

[Back to top...](#top)

## Conclusion: <a class = "anchor" id = "end"></a>

[Back to top...](#top)